# Introduction to Ray AI Runtime (AIR)
---
(*Suggested Time to Complete: 30 minutes*)

✨ Welcome to Part II of "Introduction to Ray"! 🪩

![Map of Ray](images/map.png)

*Figure 1*

Ray AI Runtime (AIR) is a unified set of libraries built on top of Ray for distributed data processing, model training, tuning, model serving, and reinforcement learning, all in Python. AIR provides simple scalable machine learning for individual workloads and end-to-end workflows, bringing together an ever-growing ecosystem of integrations with your favorite machine learning frameworks.

Before we lay out each library and their unique jobs to be done, let's take a moment to motivate Ray AIR by taking a high-level view of the typical data science and machine learning workflow. Developing a machine learning system is an iterative and often cyclical process that touches on the following stages:

1. Data Collection & Feature Engineering: source, sample, and label raw data; preprocess raw data into well-defined input dataset(s)
2. Model Training: the learning part of machine learning that could utilize a popular framework like PyTorch, XGBoost, or Tensorflow
3. Hyperparameter Tuning: improve upon your baseline model by searching a hyperparameter space
3. Model Evaluation: perform batch inference on new data to evaluate perforamnce, potentially triggering more feature engineering or finding a more relevant set of data
4. Deployment: deploy your solution to production and/or serve your model to the end user

Each of the five native libraries that Ray AIR wraps tackles a piece of the ML specific tasks outlined above that you can see illustrated in *Figure 2*. Because this abstraction layer is built on top of Ray Core, it is distributed by nature.

1. 📊 [Ray Data](https://docs.ray.io/en/latest/data/dataset.html): scalable, framework-agnostic loading and transforming raw data across training and prediction
2. 🚂 [Ray Train](https://docs.ray.io/en/latest/train/train.html): distributed multi-node model training with fault tolerance that integrates with your favorite training libraries
3. 📈 [Ray Tune](https://docs.ray.io/en/latest/tune/index.html): scales experiment execution and hyperparameter tuning to optimize model performance
4. 🍦 [Ray Serve](https://docs.ray.io/en/latest/serve/index.html): deploys your model for online inference, with optional microbatching to improve performance
5. 🦾 [Ray RLlib](https://docs.ray.io/en/latest/rllib/index.html): distributed reinforcement learning workloads that integrate with the other Ray AIR libraries above

In this module, we will contextualize Ray Data, Train, Tune, and Serve with a common ML pipeline and discuss how each library facilitates the distinct steps we need to distribute an end-to-end example. Then, we will look at scaling individual workloads with a reinforcement learning specific application for RLlib.

**Learning Objectives**
1. Understand the high-level data science libraries that compose Ray AIR: Data, Train, Tune, Serve, and RLlib
2. Understand how to use Ray AIR as a unified toolkit to write an individual as well as end-to-end ML application in Python
3. Practice key concepts from each stage of the ML pipeline
    - Data - use out-of-the-box `Preprocessor`s to load and transform data
    - Train - use AIR `Trainer`s for supported ML frameworks
    - Tune - use AIR `Tuner`s for hyperparameter search
    - BatchPredictor - use AIR `BatchPredictor` to load model from best checkpoint for batch inference
    - Serve - use `PredictorDeployment` for online inference
    - RLlib - distribute RL workloads with RLlib

**Prerequisites**
- [Introduction to Ray Notebook](https://github.com): introduces Ray as a low-level distributed computing framework and covers key elements of Ray Core

![End to End](images/e2e_air.png)

*Figure 2*

# Predicting Big Tips w/ NYC Taxi Data
***

To illustrate Ray AIR's capabilities, we will walk through an end-to-end example, building a simple machine learning pipeline using Ray Data, Train, Tune, and Serve. In each section, we will introduce each library's key concepts and how they relate to other libraries in AIR. Then, we will apply our learnings in a structured example and an open-ended excercise that relates back to the problem narrative.

Suppose we ************ we want to make an application for taxi drivers so that they can try to maximize their tips, so we want to predict big tips depending on features coming in.


To illustrate Ray AIR's capabilities, we will walk through a practical, end-to-end example of building a typical machine learning pipeline using Ray Data, Train, Tune, and Serve. We will build a binary classification model based off the [New York City Taxi Dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page) which contains over 2 million records of yellow cab rides taken in the city. We're trying to train out model to predict if rides will result in a  large tip (<20%) based off of features such as the ride duration, date, time, overall fare amount, and passenger count. Our workflow will start by loading data, performing some basic preprocessing, train the model with XGBoost, tune hyperparameters, perform batch inference, and then finally deploy our application online.

## 1. Ray Data
***
First up, we want to load in the taxi dataset and transform its raw data into features that will be input to our machine learning model. Ray Datasets are the standard way to load and exchange data in Ray libraries and applications. They provide basic distributed data transformations such as map, filter, and repartition and are compatible with a variety of file formats, data sources, and distributed frameworks.

**Key Concepts**

`Dataset`: The standard way to load and exchange data in Ray AIR. In AIR, Datasets are used extensively for data loading, preprocessing, and batch inference.

`Preprocessors`: Preprocessors are primitives that can be used to transform input data into features. Preprocessors operate on Datasets, which makes them scalable and compatible with a variety of datasources and dataframe libraries. A Preprocessor is fitted during Training, and applied at runtime in both Training and Serving on data batches in the same way. AIR comes with a collection of built-in preprocessors, and you can also define your own with simple templates which you can read more about in our [User Guide](https://docs.ray.io/en/latest/ray-air/preprocessors.html).

![Ray Data Code Snippet](images/data_code.png)

*Figure 3*

### Import Relevant Packages + Starting Ray
To start, we'll import Ray (check out our [installation instructions](https://docs.ray.io/en/latest/ray-overview/installation.html)) and start a Ray cluster on our machine that can utilize all the cores available to you as workers. We use `ray.is_initialized` to ensure that we only have one Ray cluster active.

In [1]:
import ray

if ray.is_initialized:
    ray.shutdown()

ray.init()

2022-10-19 21:19:17,442	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.6
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8265


### Create Ray Dataset
Here, we read in the data from an S3 `.csv` datasource. 

In [ ]:
# we use the June 2021 dataset for training and the June 2022 dataset for batch inference later
dataset = ray.data.read_parquet("data/nyc_taxi_2021.parquet")

# split data into training and validation subsets
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)
valid_dataset = valid_dataset.drop_columns(["is_big_tip"])

# repartition the dataset for maximum parallelism
train_dataset.repartition(100)
valid_dataset.repartition(100)

**👾 Coding Excercise 👾**

There exist many [`Dataset` API elements](https://docs.ray.io/en/latest/data/api/dataset.html#) available for common transformations and operations. Using the above as a reference:
1. inspect the schema
2. visualize the first five samples
3. evaluate whether this dataset contains extreme outliers; if so, drop them

In [ ]:
### YOUR CODE HERE ###

### Preprocessing
In the code below, we define a `StandardScaler` preprocessor because we wish to normalize the *approximately normal* `trip_distance` and `trip_duration` columns.

What's nice about a Ray AIR `Preprocessor` is that it is automatically incorporated into an end-to-end application built with AIR.

- During Training: `Preprocessor` is passed into a `Trainer` to `fit` and `transform` input `Dataset`s.
- During Tuning: each `Trial` will isntantiate its own copy of the `Preprocessor` and the fitting and transofrmation logic will occur once per `Trial`
- During Checkpoint: the `Preprocessor` is saved in the `Checkpoint` is if was passed into the `Trainer`
- During Predicting: if the `Checkpoint` contains a `Preprocessor`, then it will be used to call `transform_batch` on input batches prior to performing inference

This kind of heavy lifting behind the scenes is one of the major advantages of Ray AIR.

In [ ]:
from ray.data.preprocessors import StandardScaler

# create a preprocessor to scale some columns
preprocessor = StandardScaler(columns=["trip_distance", "trip_duration"])

## 2. Ray Train
***
Following data preprocessing, we can move forward with defining our model and fitting it on our training set. Ray Train is a lightweight library for distributed deep learning that allows you to easily supercharge your distributed training on Ray.

**Key Concept**

`Trainer`: Trainers are wrapper classes around third-party training frameworks such as XGBoost, Pytorch, and Tensorflow. They are built to help integrate with core Ray Actors (for distribution), Ray Tune, and Ray Datasets.

![Ray Train Code Snippet](images/train_code.png)

*Figure 3*

### Define AIR `Trainer`

In Ray AIR, `Trainer`s provide a way to scale out training with popular machine learning frameworks. As part of Ray Train, `Trainer`s enable users to run distributed multi-node training with fault tolerance. Fully integrated with the Ray ecosystem, Trainers leverage Ray Data to enable scalable preprocessing and performant distributed data ingestion. Also, Trainers can be composed with `Tuner`s for distributed hyperparameter tuning. After executing training, `Trainer`s output the trained model in the form of a `Checkpoint`, which can be used for batch or online prediction inference.

There are three broad categories of Trainers that AIR offers:

- Deep Learning Trainers (Pytorch, Tensorflow, Horovod)
- Tree-based Trainers (XGBoost, LightGBM)
- Other ML frameworks (HuggingFace, Scikit-Learn, RLlib)

In the example below, we will use an `XGBoostTrainer`to perform binary classification on these NYC Taxi rides. To construct a `Trainer`, you extent drom the `BaseTrainer` interface by providing:

- a `scaling_config` which specifies how many parallel training workers and what type of resources (CPUs/GPUs) to use per worker during training.
- a `run_config` which configures a variety of runtime parameters such as fault tolerance, logging, and callbacks
- a collection of datasets and a preprocessor for the provided datasets which configures preprocessing and the datasets to ingest from
- `resume_from_checkpoint` which is a checkpoint path to resume from, should your training run be interrupted

In summary, the steps for our example are:

1. define the parallelism for Ray compute
2. define the XGBoost parameters for training
3. supply the preprocessor for fitting and transforming dataset during training and validation
4. provide the datasets for training and validation
5. invoke `trainer.fit` to fit the model

In [ ]:
from ray.air.config import ScalingConfig
from ray.train.xgboost import XGBoostTrainer

trainer = XGBoostTrainer(
    scaling_config=ScalingConfig(
        # Number of workers to use for data parallelism.
        num_workers=2,
        # Whether to use GPU acceleration.
        use_gpu=False),
    label_column="is_big_tip",
    num_boost_round=20,
    params={
        # XGBoost specific params
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    # our train and validation dataset and preprocessor
    datasets={"train": train_dataset, "valid": valid_dataset},
    preprocessor=preprocessor
)

Trainer objects produce a Result object after calling `.fit()`. These objects contain training metrics as well as checkpoints to retrieve the best model.

In [ ]:
# fit the trainer
result = trainer.fit()

**👾 Coding Excercise 👾**

There exist many [`Dataset` API elements](https://docs.ray.io/en/latest/data/api/dataset.html#) available for common transformations and operations. Using the above as a reference:
1. inspect the schema
2. visualize the first five samples
3. evaluate whether this dataset contains extreme outliers; if so, drop them

In [ ]:
### YOUR CODE HERE ###

## 3. Ray Tune
***
Ray Tune is a Python library for fast hyperparameter tuning at scale. Easily distribute your trial runs to quickly find the best hyperparameters.

**Key Concept**

`Tuner`: Tuners offer scalable hyperparameter tuning as part of Ray Tune. Tuners can work seamlessly with any Trainer but also can support arbitrary training functions.

### Use AIR `Tuner` for Hyperparameter Search
What if you want to do hyperparameter optimization during training and use the best config for the model? Well, you can then use Tuner and supply your training function, `Trainer`, as part of the argument, along with other Tuner configuration.

1. define the hyperparameter space
2. define `TuneConfig` for number of trials and parallelism
3. invoke `tuner.fit()`

![Ray Tune Code Snippet](images/tune_code.png)

*Figure 5*

In [ ]:
from ray import tune

param_space = {"params": {"max_depth": tune.randint(1, 9)}}
metric = "train-logloss"
our_mode="min"

In [ ]:
from ray.tune.tuner import Tuner, TuneConfig
from ray.air.config import RunConfig

tuner = Tuner(
    trainer,
    param_space=param_space,
    tune_config=TuneConfig(num_samples=5, metric=metric, mode=our_mode),
)
# Execute tuning.
result_grid = tuner.fit()

## 4. Ray AIR Checkpoints
***
The AIR trainers, tuners, and custom pretrained models generate Checkpoints. An AIR Checkpoint is a format for models that are used across different components of the Ray AI Runtime. This common format allows easy interoperability among AIR components and seamless integration with external supported machine learning frameworks.

**Key Concept**

`Checkpoints`: The AIR trainers, tuners, and custom pretrained model generate a framework-specific Checkpoint object. Checkpoints are a common interface for models that are used across different AIR components and libraries.

`BatchPredictor`: loads the best model from a checkpoint to perform batch inference

![Batch Predictor Code Snippet](images/batchpredict_code.png)

*Figure 6*

### Use AIR `BatchPredictor` for Batch Prediction
Once you have trained and tuned your model, create a batch prdictor from best model using the `best_result.checkpoint` and do batch inference.

In [ ]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.xgboost import XGBoostPredictor

# I need a best_result

batch_predictor = BatchPredictor.from_checkpoint(best_result.checkpoint, XGBoostPredictor)

test_dataset = ray.data.read_parquet("data/nyc_taxi_2022.parquet").drop("is_big_tip")

predicted_probabilities = batch_predictor.predict(test_dataset)
print("PREDICTED PROBABILITIES")
predicted_probabilities.show()

## 5. Ray Serve
***
Ray Serve lets you serve machine learning models in real-time or batch using a simple Python API. Serve individual models or create composite model pipelines, where you can independently deploy, update, and scale individual components.

**Key Concept**

`Deployments`: Deploy the model as an inference service by using Ray Serve and the `PredictorDeployment` class.

![Ray Serve Code Snippet](images/serve_code.png)

*Figure 7*

### Use `PredictorDeployment` for Online Inference
Deploy the best model as an inference service by using Ray Serve and the `PredictorDeployment` class. After deploying the service, you can send requests to it.

In [ ]:
from ray import serve
from fastapi import Request
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import pandas_read_json

serve.run(
    PredictorDeployment.options(name="XGBoostService", num_replicas=2, route_prefix="/rayair").bind(
        XGBoostPredictor, result.checkpoint, http_adapter=pandas_read_json
    )
)

In [ ]:
import requests

sample_input = test_dataset.take(1)
sample_input = dict(sample_input[0])

output = requests.post("http://localhost:8000/rayair", json=[sample_input]).json()
print(output)

In [ ]:
ray.shutdown()

## Summary
You've now just created a Ray Dataset, preprocessed some features, built a model with XGBoost, searched a hyperparameter space for the best configuration, loaded the best model from a checkpoint to perform batch inference, and served that model for online inference. Through this end-to-end example, you explored how to use Ray AIR to distribute an entire ML pipeline.

### Key Concepts

- `Datasets`
- `Preprocessors`
- `Trainers`
- `Tuner`
- `Checkpoints`
- `BatchPredictor`
- `Deployments`

### Next Up

Now that you've seen how you can use Ray AIR's unified toolkit to scale an end-to-end machine learning application, let's see how we can use it to scale individual workloads. In the next section we will cover a reinforcement learning example

# Reinforcement Learning on Ray AIR
In this example, we're going to train a reinforcement learning agent using online training. Online training means that the data from the environment is sampled while we are running the algorithm. In contrast, offline training uses data that has been stored on disk before.

## Ray RLLib
RLlib is the industry-standard reinforcement learning Python framework built on Ray. Designed for quick interation and a fast path to production, it includes 25+ latest algorithms that are all implemented to run at scale and in multi-agent mode.

RLlib is an open-source library for reinforcement learning (RL), offering support for production-level, highly distributed RL workloads while maintaining unified and simple APIs for a large variety of industry applications. Whether you would like to train your agents in a multi-agent setup, purely from offline (historic) datasets, or using externally connected simulators, RLlib offers a simple solution for each of your decision making needs.

If you either have your problem coded (in Python) as an RL environment or own lots of pre-recorded, historical behavioral data to learn from, you will be up and running in only a few days. RLlib is already used in production by industry leaders in many different verticals such as climate control, industrial control, manufacturing an dlogistics, finance, gaming, automobile, robotics, boat design, and many others.

We can start by running some imports. We're using OpenAI's gym, which is a standard API for reinforcement learning.

In [ ]:
import gym
import numpy

In [ ]:
from ray.air import RunConfig
from ray.air import ScalingConfig
from ray.air import Checkpoint

from ray.train.rl import RLTrainer
from ray.train.rl import RLPredictor

from ray.air import Result

from ray.tune import Tuner

from ray.rllib.algorithms.marwil import BCTrainer

We're going to use the CartPole environment. insert a gif of cartpole as well as description of the premise

In [ ]:
env = gym.make("CartPole-v0")

Set up an RL Trainer??

In [ ]:
trainer = RLTrainer(
    run_config = RunConfig(stop={"training_iteration": 5}),
    scaling_config = ScalingConfig(num_workers=2, use_gpu=False),
    algorithm="PPO",
    config={
        "env": "CartPole-v1",
        "framework": "tf",
    },
)

In [ ]:
def train_rl_ppo_online(num_workers: int, use_gpu: bool = False) -> Result:
    print("Starting online training")
    trainer = RLTrainer(
        run_config=RunConfig(stop={"training_iteration": 5}),
        scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
        algorithm="PPO",
        config={
            "env": "CartPole-v1",
            "framework": "tf",
        },
    )
    # Todo (krfricke/xwjiang): Enable checkpoint config in RunConfig
    # result = trainer.fit()
    tuner = Tuner(
        trainer,
        _tuner_kwargs={"checkpoint_at_end": True},
    )
    result = tuner.fit()[0]
    return result

In [ ]:
def evaluate_using_checkpoint(checkpoint: Checkpoint, num_episodes) -> list:
    predictor = RLPredictor.from_checkpoint(checkpoint)

    env = gym.make("CartPole-v0")

    rewards = []
    for i in range(num_episodes):
        obs = env.reset()
        reward = 0.0
        done = False
        while not done:
            action = predictor.predict(np.array([obs]))
            obs, r, done, _ = env.step(action[0])
            reward += r
        rewards.append(reward)

    return rewards

In [ ]:
result = train_rl_ppo_online(num_workers=2, use_gpu=False)

In [ ]:
num_eval_episodes = 3

rewards = evaluate_using_checkpoint(result.checkpoint, num_episodes=num_eval_episodes)
print(f"Average reward over {num_eval_episodes} episodes: " f"{np.mean(rewards)}")

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

# Extra Resources
---
If you would like to practice your new skills further with some in-depth examples beyond the embedded coding excercises, take a look at this list of suggested problems:
- Watch the Ray Summit Talk on [Introduction to Ray AIR](https://github.com/ray-project/hackathon5-algo)
- Check out the [Ray AIR Documentation](https://docs.ray.io/en/latest/ray-air/getting-started.html)
- Understand its [Components and APIs](https://docs.ray.io/en/latest/ray-air/package-ref.html)
- Ray AIR [User Guides](https://docs.ray.io/en/latest/ray-air/user-guides.html) and [Examples](https://docs.ray.io/en/latest/ray-air/examples/index.html)


# Next Steps
---
🎉 Congratulations! You have completed the tutorial on an Introduction to Ray AI Runtime! We dicussed each library in Ray AIR (Data, Train, Tune, Serve, RLLib) and saw some example machine learning workloads to be done with each. In the next module, we will introduce the ecosystem of integrated libraries runs on Ray Core's distributed execution engine, and with Ray Clusters, you can deploy your workloads on AWS, GCP, Azure, or on Kubernetes.

From here, you can learn and get more involved with our active community of developers and researchers by checking out the following resources:
- 💻 [Official Ray Website](https://www.ray.io/): Browse the ecosystem and use this site as a hub to get the information that you need to get going and building with Ray.
- 💬 [Join the Community on Slack](https://forms.gle/9TSdDYUgxYs8SA9e8): Find friends to discuss your new learnings in our Slack space.
- 📣 [Use the Discussion Board](https://discuss.ray.io/): Ask questions, follow topics, and view announcements on this community forum.
- 🙋‍♀️ [Join a Meetup Group](https://www.meetup.com/Bay-Area-Ray-Meetup/): Tune in on meet-ups to listen to compelling talks, get to know other users, and meet the team behind Ray.
- 🪲 [Open an Issue](https://github.com/ray-project/ray/issues/new/choose): Ray is constantly evolving to improve developer experience. Submit feature requests, bug-reports, and get help via GitHub issues.